# Ejercicio Extra GPU. Alumno05. 

In [11]:
import numpy as np

def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

#value = int(sys.argv[1])
value = 5*10**6

X = np.random.rand(value)


# Utilizando las operaciones mágicas de ipython

tiempo = %timeit -r 2 -o -q reduc_operation(X)

print(f"Tiempo que tarda la función reduc_operation con un array numpy de {value} valores:", tiempo)
print(f"Resultado: {reduc_operation(X)}\n")


# Utilizando numpy.sum()

tiempo = %timeit -r 2 -o -q np.sum(X)

print("Time taken by reduction operation using numpy.sum():", tiempo)
print("Now, the result using numpy.sum():", np.sum(X),"\n ")


# Utilizando numpy.ndarray.sum()

tiempo= %timeit -r 2 -o -q X.sum()

print("Time taken by reduction operation using numpy.ndarray.sum():", tiempo)
print("Now, the result using numpy.ndarray.sum():", X.sum())


Tiempo que tarda la función reduc_operation con un array numpy de 5000000 valores: 494 ms ± 2.68 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Resultado: 2501656.4808689193

Time taken by reduction operation using numpy.sum(): 1.83 ms ± 3.14 µs per loop (mean ± std. dev. of 2 runs, 1,000 loops each)
Now, the result using numpy.sum(): 2501656.480869037 
 
Time taken by reduction operation using numpy.ndarray.sum(): 1.82 ms ± 1.82 µs per loop (mean ± std. dev. of 2 runs, 1,000 loops each)
Now, the result using numpy.ndarray.sum(): 2501656.480869037


#### 1. Multiprocessing con más de un parámetro: starmap. 

In [10]:
import numpy as np
from multiprocessing import Pool
import time

# Modificar la función para recibir el rango de inicio y fin
def sum_multiprocessing(A, ini, fin):
    """Compute the sum of the elements of Array A in the range [ini, fin)."""
    s = 0
    for i in range(ini, fin):
        s += A[i]
    return s

# Crear el array de prueba
value = 5 * 10**6
X = np.random.rand(value)

# Crear una función para dividir la suma en procesos
def suma_starmap(array, ranges):
    """Divide la operación de suma en procesos y combina los resultados."""
    with Pool() as pool:
        resultados_parciales = pool.starmap(sum_multiprocessing, [(array, start, end) for start, end in ranges])
    return sum(resultados_parciales)

# Suma con un solo proceso
ranges_1 = [(0, value)]

inicio=time.time()
result_1 = suma_starmap(X, ranges_1)
fin=time.time()
print(f"Tiempo con un proceso: {fin-inicio} segundos. Resultado: {result_1}")

# Suma con dos procesos
ranges_2 = [(0, value // 2), (value // 2, value)]

inicio=time.time()
result_2 = suma_starmap(X, ranges_2)
fin=time.time()
print(f"Tiempo con dos procesos: {fin-inicio} segundos. Resultado: {result_2}")

# Suma con cuatro procesos
ranges_4 = [
    (0, value // 4),
    (value // 4, value // 2),
    (value // 2, 3 * value // 4),
    (3 * value // 4, value),
]

inicio=time.time()
result_4 = suma_starmap(X, ranges_4)
fin=time.time()
print(f"Tiempo con cuatro procesos: {fin-inicio} segundos. Resultado: {result_4}")


Tiempo con un proceso: 0.6049990653991699. Resultado: 2500260.1136280205
Tiempo con dos procesos: 0.36820006370544434. Resultado: 2500260.1136276154
Tiempo con cuatro procesos: 0.33579468727111816. Resultado: 2500260.1136276103


Usando starmap para hacer la misma segmentación que realizamos en el apartado 3.2, vemos los mismos resultados: conforme segmentamos el array para procesar el cálculo en hilos paralelos, se acelera el programa. 

#### 2. Usando CuPy de forma alternativa al ejercicio 3.3.
En mi ejercicio 3.3 ya he utilizado CuPy cargando en la GPU el array previamente creado, como se pide en este enunciado. Por tanto, complementaré haciendo lo opuesto: repitiendo el proceso creando un array cupy desde 0. 

In [13]:
# Movilizo el array numpy "X" a la GPU como un array CuPy
import cupy as cp

# array cupy creado desde 0. 
Y = cp.random.rand(value)

# Al usar la GPU, no se puede medir el tiempo con las funciones mágicas de ipython. Se debe usar la librería time, y sincronizar. 

# Utilizando cupy.sum()
cp.cuda.Stream.null.synchronize()
inicio = time.time()
resultado= cp.sum(Y)
cp.cuda.Stream.null.synchronize()
fin = time.time()

print("Tiempo usando cupy.sum():", (fin-inicio)*1000, "ms")
print("Resultado usando cupy.sum():",resultado,"\n ")


# Utilizando cupy.ndarray.sum()
cp.cuda.Stream.null.synchronize()
inicio = time.time()
resultado= Y.sum()
cp.cuda.Stream.null.synchronize()
fin = time.time()

print("Tiempo usando cupy.ndarray.sum() sobre el array X alojado en GPU:", (fin-inicio)*1000, "ms")
print("Resultado:", resultado)


Tiempo usando cupy.sum(): 72.69906997680664 ms
Resultado usando cupy.sum(): 2500014.9130936987 
 
Tiempo usando cupy.ndarray.sum() sobre el array X alojado en GPU: 0.25272369384765625 ms
Resultado: 2500014.9130936987
